# NNI量化实践

量化通过减少表示权重或激活所需的位数来减小模型大小并加快推理时间。在 NNI 中，支持训练后量化算法和量化感知训练算法。本节使用训练后量化算法为例来展示NNI中量化的用法。

> 参考文档：https://nni.readthedocs.io/zh/stable/tutorials/quantization_quick_start.html

In [70]:
import random
import time
import numpy as np
import torch
from matplotlib import pyplot as plt
from torch import nn
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchvision.transforms import *
from tqdm.auto import tqdm
import torch.nn.functional as F
from torchvision import datasets

In [71]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

## 加载model

In [78]:
# 设置归一化
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# 获取数据集
train_dataset = datasets.MNIST(root='../../ch02/data/mnist', train=True, download=True, transform=transform)  
test_dataset = datasets.MNIST(root='../../ch02/data/mnist', train=False, download=True, transform=transform)  # train=True训练集，=False测试集
total_test_samples = len(test_dataset)
# 设置DataLoader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [73]:
# 定义一个LeNet网络
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=16 * 4 * 4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.fc3 = nn.Linear(in_features=84, out_features=num_classes)
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.maxpool(self.relu1(self.conv1(x)))
        x = self.maxpool(self.relu2(self.conv2(x)))

        x = x.view(x.size()[0], -1)
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)

        return x
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model = LeNet().to(device=device)

In [74]:
def training_step(batch, model):
    x, y = batch[0].to(device), batch[1].to(device)
    logits = model(x)
    criterion = nn.CrossEntropyLoss() 
    loss = criterion(logits, y)
    return loss

def training_model(model: torch.nn.Module, optimizer, training_step, scheduler,
                   max_steps, max_epochs):
    model.train()
    max_epochs = max_epochs if max_epochs else 1 if max_steps is None else 100
    current_steps = 0

    # training
    for epoch in range(max_epochs):
        print(f'Epoch {epoch} start!')
        for batch in train_loader:
            optimizer.zero_grad()
            loss = training_step(batch, model)
            loss.backward()
            optimizer.step()
            current_steps += 1
            if max_steps and current_steps == max_steps:
                return
        if scheduler is not None:
            scheduler.step()

In [79]:
def evaluating_model(model: torch.nn.Module):
    model.eval()
    # testing
    correct = 0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device), y.to(device)
            logits = model(x)
            preds = torch.argmax(logits, dim=1)
            correct += preds.eq(y.view_as(preds)).sum().item()
    return correct / total_test_samples

In [80]:
optimizer = SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)

start = time.time()
training_model(model, optimizer, training_step, None, None, 5)
print(f'pure training 5 epochs: {time.time() - start}s')
start = time.time()
acc = evaluating_model(model)
print(f'pure evaluating: {time.time() - start}s    Acc.: {acc}')

Epoch 0 start!
Epoch 1 start!
Epoch 2 start!
Epoch 3 start!
Epoch 4 start!
pure training 5 epochs: 110.96141934394836s
pure evaluating: 1.8336389064788818s    Acc.: 0.9895


## 模型量化


In [77]:
import nni
from nni.compression.quantization import QATQuantizer
from nni.compression.utils import TorchEvaluator


optimizer = nni.trace(SGD)(model.parameters(), lr=0.01, momentum=0.9, weight_decay=5e-4)
evaluator = TorchEvaluator(training_model, optimizer, training_step) 

# 将conv、fc、relu层量化为int8
config_list = [{
    'op_names': ['conv1', 'conv2', 'fc1', 'fc2'],
    'target_names': ['_input_', 'weight', '_output_'],
    'quant_dtype': 'int8',
    'quant_scheme': 'affine',
    'granularity': 'default',
},{
    'op_names': ['relu1', 'relu2'],
    'target_names': ['_output_'],
    'quant_dtype': 'int8',
    'quant_scheme': 'affine',
    'granularity': 'default',
}]

quantizer = QATQuantizer(model, config_list, evaluator, len(train_loader))
real_input = next(iter(train_loader))[0].to(device)
quantizer.track_forward(real_input)

start = time.time()
_, calibration_config = quantizer.compress(None, max_epochs=5)
print(f'pure training 5 epochs: {time.time() - start}s')

print(calibration_config)
start = time.time()
acc = evaluating_model(model)
print(f'quantization evaluating: {time.time() - start}s    Acc.: {acc}')

Epoch 0 start!
Epoch 1 start!
Epoch 2 start!
Epoch 3 start!
Epoch 4 start!
pure training 5 epochs: 101.41740226745605s
defaultdict(<class 'dict'>, {'fc1': {'weight': {'scale': tensor(0.0021), 'zero_point': tensor(22.), 'quant_dtype': 'int8', 'quant_scheme': 'affine', 'quant_bits': 8, 'tracked_max': tensor(0.2163), 'tracked_min': tensor(-0.3066)}, '_input_0': {'scale': tensor(0.0655), 'zero_point': tensor(-127.), 'quant_dtype': 'int8', 'quant_scheme': 'affine', 'quant_bits': 8, 'tracked_max': tensor(16.6477), 'tracked_min': tensor(0.)}, '_output_0': {'scale': tensor(0.1609), 'zero_point': tensor(7.), 'quant_dtype': 'int8', 'quant_scheme': 'affine', 'quant_bits': 8, 'tracked_max': tensor(19.3163), 'tracked_min': tensor(-21.5482)}}, 'conv2': {'weight': {'scale': tensor(0.0033), 'zero_point': tensor(-15.), 'quant_dtype': 'int8', 'quant_scheme': 'affine', 'quant_bits': 8, 'tracked_max': tensor(0.4663), 'tracked_min': tensor(-0.3678)}, '_input_0': {'scale': tensor(0.0432), 'zero_point': tens

eval:   0%|          | 0/10 [00:00<?, ?it/s]

quantization evaluating: 1.7575781345367432s    Acc.: 0.9851


> 量化后虽然训练需要的时间增加了，但推理的时间减少了